<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.6

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

#class instantiations
ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = dl.MulticlassOutput(load_breast_cancer()['target'])
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
uploaded = files.upload()
data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
data.extractall()

Saving IMGDS1.zip to IMGDS1.zip


In [4]:
#image directories... if any
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']

# Parameter Grid Templates

rnn_grid = {'output_dim': [int], 'embedding': [vocab], 'nodes': [int], 'activation': [str], 
              'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'optimizer': [str], 
              'method': [str], 'bidirectional': [bool]} 

cnn_grid = {'output_dim': [int], 'base_filters':[int], 'kernel_size': [int], 'activation': [str], 
            'nblocks': [int], 'pool':[int], 'dropout': [bool], 'closer': [bool], 'optimizer': [str], 
            'metrics': [str]}

tc_grid = {'output_dim': [int], 'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 
               'dropout': [bool], 'nlayers': [int], 'closer': [bool], 'loss_func': [str], 'optimizer': [str]}

tr_grid = {'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'nlayers': [int], 'closer': [bool], 
                'loss': [str], 'optimizer': [str], 'y_col': [str]}

In [ ]:
def Preprocess(df, data_str, target_str):
  '''fully preprocesses a dataframe of Text Documents for classification'''
  avg = 0
  for i in range(len(df[data_str])):
    avg += len(list(df[data_str])[i].split(' '))/len(df)
  pad = int(avg)
  text, labels = dl.ModelReadyText1(df[data_str], df[target_str], pad)
  return text, labels

blogs, blog_labels = Preprocess(Wikipedia, 'Text', 'AI')
tweets, tweet_labels = Preprocess(Twitter, 'tweet', 'is_there_an_emotion_directed_at_a_brand_or_product')

In [5]:
def GetVocab(df, data_str):
  words = []
  for i in range(len(df)):
    word_lst = list(df[data_str])[i].replace('\n', ' ').split(' ')
    for word in word_lst:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  return len(np.unique(words))

In [4]:
def TCP(X, y):
  dl = DeepLearning()
  avg = 0
  words = []
  for i in range(len(X)):
    avg += len(list(X)[i].split(' '))/len(X)
    word_lst = list(X)[i].replace('\n', ' ').split(' ')
    for word in word_lst:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  pad = int(avg)
  text, labels = dl.ModelReadyText1(X, y, pad)    
  return text, labels, len(np.unique(words))

In [6]:
blogs_vocab = GetVocab(Wikipedia, 'Text')
tweets_vocab = GetVocab(Twitter, 'tweet')

In [5]:
def BigDawg(X, y, params, task, ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  dl = DeepLearning()
  if task == 'NLP':
    X2, y2, _ = TCP(X, y)
    func = dl.RNN #insert CV after this
  if task == 'CV':
    X2, _, y2, _ = dl.ModelReadyPixles(directories=X, classes=y, target_size=(ts, ts))
    func = dl.CNN
  if task == 'TC':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularClassification
  if task == 'TR':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularRegression
  try:
    return dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=5, regression=regression)
  except:
    return dl.CollectPerformance(params, func, X2, y2.reshape(-1))

In [6]:
#rnn_grid = {'output_dim': [3], 'embedding': [tweets_vocab], 'nodes': [64], 'activation': ['tanh'], 'regularizer': [None, 'L1'], 
            #'stacking': [False], 'dropout': [False], 'optimizer': ['adam'], 'method': ['GRU', 'LSTM'], 'bidirectional': [False]}

cnn_grid = {'output_dim': [2], 'base_filters':[32], 'kernel_size': [2], 'activation': ['relu'], 'nblocks': [3], 
            'pool':[2], 'dropout': [False, True], 'closer': [False], 'optimizer': ['adam', 'rmsporp'], 'metrics': ['accuracy']}

#tc_grid = {'output_dim': [2], 'nodes': [64], 'activation': ['relu'], 'regularizer': [None, 'L1'], 'stacking': [False, True], 
           #'dropout': [False], 'nlayers': [4], 'closer': [False], 'loss_func': ['binary_crossentropy'], 'optimizer': ['adam']}

#tr_grid = {'nodes': [64], 'activation': ['relu'], 'regularizer': [None, 'L1'], 'stacking': [False], 'dropout': [False], 
           #'nlayers': [4], 'closer': [False], 'loss_func': ['MSE', 'MAE'], 'optimizer': ['rmsprop'], 'y_col': ['price']}

BigDawg(directories, classes, cnn_grid, 'CV', epochs=3)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
preforming 4 total fits ...
Epoch 1/3
200/200 [==============================] - 103s 511ms/step - loss: 0.6891 - accuracy: 0.5385 - val_loss: 0.6282 - val_accuracy: 0.6444
Epoch 2/3
200/200 [==============================] - 101s 504ms/step - loss: 0.5849 - accuracy: 0.6908 - val_loss: 0.5222 - val_accuracy: 0.7419
Epoch 3/3
200/200 [==============================] - 100s 502ms/step - loss: 0.4783 - accuracy: 0.7640 - val_loss: 0.5289 - val_accuracy: 0.7350
1 of 4 fits complete!
Epoch 1/3
200/200 [==============================] - 102s 508ms/step - loss: 0.6573 - accuracy: 0.5922 - val_loss: 0.5801 - val_accuracy: 0.6944
Epoch 2/3
200/200 [===============

,output_dim,base_filters,kernel_size,activation,nblocks,pool,dropout,closer,metrics,epochs,batch_size,optimizer,accuracy,loss,val_accuracy,val_loss
0,2,32,2,relu,3,2,False,False,accuracy,3,32,adam,0.765938,0.477723,0.735000,0.528899
1,2,32,2,relu,3,2,False,False,accuracy,3,32,rmsporp,0.790781,0.444370,0.724375,0.541258
2,2,32,2,relu,3,2,True,False,accuracy,3,32,adam,0.760781,0.488392,0.763125,0.499014
3,2,32,2,relu,3,2,True,False,accuracy,3,32,rmsporp,0.739688,0.521568,0.731875,0.531596


# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']